<a href="https://colab.research.google.com/github/sujathachittiri/Dissertation/blob/main/Engine/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Generate a realistic merged regulatory-style dataset (~7000 rows) inspired by open-source financial datasets (BankSim, UCI Credit, Kaggle loans)
# and structured to resemble regulatory AnaCredit-like fields. This file is saved to /mnt/data/realistic_regulatory_data.csv
# The dataset is synthetic but contains realistic distributions, categorical diversity, and injected anomalies.
# This code is Colab/local-friendly and doesn't use ace_tools.

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random, os

random.seed(123)
np.random.seed(123)

N = 7000  # medium size dataset

# Account IDs
account_ids = [f"CUST-{500000 + i}" for i in range(N)]

# Report dates spanning 2023-2025 quarterly periods
start = datetime(2023,1,1)
report_dates = [start + timedelta(days=int(x)) for x in np.random.randint(0, 1000, size=N)]
report_dates = pd.to_datetime(report_dates).date

# Exposure amounts: mix of lognormal and a portion from loan-like distributions
exposure_amount = np.round(np.where(
    np.random.rand(N) < 0.7,
    np.random.lognormal(mean=12, sigma=1.0, size=N),          # typical exposures
    np.random.normal(loc=2000000, scale=800000, size=N)      # larger corporate exposures
)).astype(int)
exposure_amount = np.where(exposure_amount < 100, np.abs(exposure_amount) + 100, exposure_amount)

# Counterparty types and derived risk weights
counterparty_types = np.random.choice(['Bank','Corporate','Retail','Sovereign','SME'], size=N, p=[0.15,0.35,0.35,0.03,0.12])

# Country codes and currency mapping (adds realism: multiple currencies)
country_codes = np.random.choice(['GB','DE','HK','US','FR','IN','SG','ES','IT','NL'], size=N,
                                 p=[0.15,0.12,0.10,0.15,0.10,0.12,0.06,0.05,0.08,0.07])
currency_map = {'IN':'INR','GB':'GBP','DE':'EUR','HK':'HKD','US':'USD','FR':'EUR','SG':'SGD','ES':'EUR','IT':'EUR','NL':'EUR'}
currency = [currency_map[c] for c in country_codes]

# Credit score proxies (0-850) for retail/SME; NaN for corporates/sov/banks
credit_score = np.where(np.isin(counterparty_types, ['Retail','SME']),
                        np.random.normal(loc=650, scale=80, size=N).astype(int), np.nan)
credit_score = np.clip(credit_score, 300, 850)
credit_score = np.where(np.isnan(credit_score), None, credit_score)

# Tenor (months) and transaction count
tenor_months = np.random.choice([6,12,24,36,48,60,120], size=N, p=[0.05,0.1,0.2,0.25,0.2,0.15,0.05])
transaction_count = np.random.poisson(lam=4, size=N)

# Risk weight baseline by counterparty type (simplified realistic mapping)
rw_map = {'Retail': 0.35, 'Corporate': 0.5, 'Bank': 0.2, 'Sovereign': 0.0, 'SME': 0.75}
risk_weight = np.array([rw_map[ct] for ct in counterparty_types])

# Introduce some country-specific risk upweighting (e.g., EM countries higher RW)
country_up = {'IN': 1.2, 'HK': 1.0, 'US': 1.0, 'GB': 1.0, 'DE': 1.0, 'FR':1.0, 'SG':1.0, 'ES':1.0, 'IT':1.1, 'NL':1.0}
risk_weight = risk_weight * np.array([country_up[c] for c in country_codes])
# Clip risk weight to max 1.0
risk_weight = np.clip(risk_weight, 0.0, 1.0).round(2)

# Capital requirement: simple regulatory-like formula (Exposure * RW * 8%)
capital_requirement = (exposure_amount * risk_weight * 0.08).round(0).astype(int)

# Source system / origin (to mimic integration scenarios)
source_system = np.random.choice(['CoreBank','LoanServicing','Treasury','TradeRepo','ExternalFeed'], size=N, p=[0.4,0.25,0.15,0.1,0.1])

# Create DataFrame
df = pd.DataFrame({
    'Account_ID': account_ids,
    'Report_Date': report_dates,
    'Exposure_Amount': exposure_amount,
    'Risk_Weight': risk_weight,
    'Capital_Requirement': capital_requirement,
    'Country_Code': country_codes,
    'Currency': currency,
    'Counterparty_Type': counterparty_types,
    'Tenor_Months': tenor_months,
    'Transaction_Count': transaction_count,
    'Source_System': source_system,
    'Credit_Score': credit_score
})

# Add derived/regulatory-like fields (e.g., Product Type, Facility ID)
df['Product_Type'] = np.random.choice(['TermLoan','Overdraft','CashCredit','TradeFinance','Repo'], size=N, p=[0.3,0.25,0.2,0.15,0.1])
df['Facility_ID'] = ['FAC-' + str(700000 + i) for i in range(len(df))]

# Inject anomalies that reflect real-world issues (~5% anomalies)
anomaly_indices = np.random.choice(df.index, size=int(0.05 * N), replace=False)
df['Is_Anomaly'] = 0
df.loc[anomaly_indices, 'Is_Anomaly'] = 1

# Types of anomalies to inject:
# 1) Extremely large exposure spike (corporate/bank) for some anomalies
spike_idx = anomaly_indices[:int(0.35*len(anomaly_indices))]
df.loc[spike_idx, 'Exposure_Amount'] = df.loc[spike_idx, 'Exposure_Amount'] * np.random.randint(10,80, size=len(spike_idx))
df.loc[spike_idx, 'Capital_Requirement'] = (df.loc[spike_idx, 'Exposure_Amount'] * df.loc[spike_idx, 'Risk_Weight'] * 0.08).round(0).astype(int)

# 2) Invalid currency codes for some anomalies
invcur_idx = anomaly_indices[int(0.35*len(anomaly_indices)):int(0.6*len(anomaly_indices))]
df.loc[invcur_idx, 'Currency'] = np.random.choice(['XXX','ZZZ','N/A'], size=len(invcur_idx))

# 3) Negative exposures (data-entry error)
neg_idx = anomaly_indices[int(0.6*len(anomaly_indices)):int(0.8*len(anomaly_indices))]
df.loc[neg_idx, 'Exposure_Amount'] = -abs(df.loc[neg_idx, 'Exposure_Amount'])
df.loc[neg_idx, 'Capital_Requirement'] = (df.loc[neg_idx, 'Exposure_Amount'] * df.loc[neg_idx, 'Risk_Weight'] * 0.08).round(0).astype(int)

# 4) Missing critical fields for some anomalies
miss_idx = anomaly_indices[int(0.8*len(anomaly_indices)):]
df.loc[miss_idx, np.random.choice(['Currency','Risk_Weight','Country_Code'], size=len(miss_idx))] = None

# Duplicate some rows to emulate ingestion duplicates (~1% extra duplicates)
dup_idx = np.random.choice(df.index, size=int(0.01 * N), replace=False)
df = pd.concat([df, df.loc[dup_idx]], ignore_index=True).reset_index(drop=True)

# Recompute flags
df['Missing_Flag'] = df.isnull().any(axis=1).astype(int)
df['Duplicate_Flag'] = df.duplicated(subset=['Account_ID','Report_Date','Exposure_Amount'], keep=False).astype(int)

# Basic profiling summary
N_total = len(df)
profile = {
    'Total_Records': N_total,
    'Total_Accounts': df['Account_ID'].nunique(),
    'Missing_Percentage': round(df['Missing_Flag'].sum() / N_total * 100, 3),
    'Duplicate_Percentage': round(df['Duplicate_Flag'].sum() / N_total * 100, 3),
    'Labeled_Anomalies': int(df['Is_Anomaly'].sum()),
    'Negative_Exposures': int((df['Exposure_Amount'] <= 0).sum())
}

# Save CSV
os.makedirs('/mnt/data', exist_ok=True)
file_path = '/mnt/data/realistic_regulatory_data.csv'
df.to_csv(file_path, index=False)

# Show profile and sample
print("Realistic regulatory-style dataset generated with {} records.".format(N_total))
print("Saved to:", file_path)
print("\nBasic Data Profile:")
for k,v in profile.items():
    print(f"{k}: {v}")

# Display top rows
df.head(10).to_csv('/mnt/data/realistic_sample_top10.csv', index=False)
file_path


Realistic regulatory-style dataset generated with 7070 records.
Saved to: /mnt/data/realistic_regulatory_data.csv

Basic Data Profile:
Total_Records: 7070
Total_Accounts: 7000
Missing_Percentage: 53.239
Duplicate_Percentage: 1.98
Labeled_Anomalies: 356
Negative_Exposures: 73


'/mnt/data/realistic_regulatory_data.csv'

In [2]:
# Regenerating realistic regulatory-style dataset (~7000 rows) and saving as CSV.
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random, os

random.seed(123)
np.random.seed(123)

N = 7000  # medium size dataset

# Account IDs
account_ids = [f"CUST-{500000 + i}" for i in range(N)]

# Report dates spanning 2023-2025 quarterly periods
start = datetime(2023,1,1)
report_dates = [start + timedelta(days=int(x)) for x in np.random.randint(0, 1000, size=N)]
report_dates = pd.to_datetime(report_dates).date

# Exposure amounts
exposure_amount = np.round(np.where(
    np.random.rand(N) < 0.7,
    np.random.lognormal(mean=12, sigma=1.0, size=N),
    np.random.normal(loc=2000000, scale=800000, size=N)
)).astype(int)
exposure_amount = np.where(exposure_amount < 100, np.abs(exposure_amount) + 100, exposure_amount)

# Counterparty types and derived risk weights
counterparty_types = np.random.choice(['Bank','Corporate','Retail','Sovereign','SME'], size=N, p=[0.15,0.35,0.35,0.03,0.12])

# Country codes and currency mapping
country_codes = np.random.choice(['GB','DE','HK','US','FR','IN','SG','ES','IT','NL'], size=N,
                                 p=[0.15,0.12,0.10,0.15,0.10,0.12,0.06,0.05,0.08,0.07])
currency_map = {'IN':'INR','GB':'GBP','DE':'EUR','HK':'HKD','US':'USD','FR':'EUR','SG':'SGD','ES':'EUR','IT':'EUR','NL':'EUR'}
currency = [currency_map[c] for c in country_codes]

# Credit score proxies
credit_score = np.where(np.isin(counterparty_types, ['Retail','SME']),
                        np.random.normal(loc=650, scale=80, size=N).astype(int), np.nan)
credit_score = np.clip(credit_score, 300, 850)
credit_score = np.where(np.isnan(credit_score), None, credit_score)

# Tenor and transaction count
tenor_months = np.random.choice([6,12,24,36,48,60,120], size=N, p=[0.05,0.1,0.2,0.25,0.2,0.15,0.05])
transaction_count = np.random.poisson(lam=4, size=N)

# Risk weight baseline
rw_map = {'Retail': 0.35, 'Corporate': 0.5, 'Bank': 0.2, 'Sovereign': 0.0, 'SME': 0.75}
risk_weight = np.array([rw_map[ct] for ct in counterparty_types])

# Country-specific upweighting
country_up = {'IN': 1.2, 'HK': 1.0, 'US': 1.0, 'GB': 1.0, 'DE': 1.0, 'FR':1.0, 'SG':1.0, 'ES':1.0, 'IT':1.1, 'NL':1.0}
risk_weight = risk_weight * np.array([country_up[c] for c in country_codes])
risk_weight = np.clip(risk_weight, 0.0, 1.0).round(2)

# Capital requirement
capital_requirement = (exposure_amount * risk_weight * 0.08).round(0).astype(int)

# Source system
source_system = np.random.choice(['CoreBank','LoanServicing','Treasury','TradeRepo','ExternalFeed'], size=N, p=[0.4,0.25,0.15,0.1,0.1])

# DataFrame
df = pd.DataFrame({
    'Account_ID': account_ids,
    'Report_Date': report_dates,
    'Exposure_Amount': exposure_amount,
    'Risk_Weight': risk_weight,
    'Capital_Requirement': capital_requirement,
    'Country_Code': country_codes,
    'Currency': currency,
    'Counterparty_Type': counterparty_types,
    'Tenor_Months': tenor_months,
    'Transaction_Count': transaction_count,
    'Source_System': source_system,
    'Credit_Score': credit_score
})

df['Product_Type'] = np.random.choice(['TermLoan','Overdraft','CashCredit','TradeFinance','Repo'], size=N, p=[0.3,0.25,0.2,0.15,0.1])
df['Facility_ID'] = ['FAC-' + str(700000 + i) for i in range(len(df))]

# Inject anomalies (~5%)
anomaly_indices = np.random.choice(df.index, size=int(0.05 * N), replace=False)
df['Is_Anomaly'] = 0
df.loc[anomaly_indices, 'Is_Anomaly'] = 1

# 1) Exposure spikes
spike_idx = anomaly_indices[:int(0.35*len(anomaly_indices))]
df.loc[spike_idx, 'Exposure_Amount'] = df.loc[spike_idx, 'Exposure_Amount'] * np.random.randint(10,80, size=len(spike_idx))
df.loc[spike_idx, 'Capital_Requirement'] = (df.loc[spike_idx, 'Exposure_Amount'] * df.loc[spike_idx, 'Risk_Weight'] * 0.08).round(0).astype(int)

# 2) Invalid currency codes
invcur_idx = anomaly_indices[int(0.35*len(anomaly_indices)):int(0.6*len(anomaly_indices))]
df.loc[invcur_idx, 'Currency'] = np.random.choice(['XXX','ZZZ','N/A'], size=len(invcur_idx))

# 3) Negative exposures
neg_idx = anomaly_indices[int(0.6*len(anomaly_indices)):int(0.8*len(anomaly_indices))]
df.loc[neg_idx, 'Exposure_Amount'] = -abs(df.loc[neg_idx, 'Exposure_Amount'])
df.loc[neg_idx, 'Capital_Requirement'] = (df.loc[neg_idx, 'Exposure_Amount'] * df.loc[neg_idx, 'Risk_Weight'] * 0.08).round(0).astype(int)

# 4) Missing critical fields
miss_idx = anomaly_indices[int(0.8*len(anomaly_indices)):]
cols_for_missing = ['Currency','Risk_Weight','Country_Code']
for i, idx in enumerate(miss_idx):
    col = cols_for_missing[i % len(cols_for_missing)]
    df.at[idx, col] = None

# Duplicates (~1%)
dup_idx = np.random.choice(df.index, size=int(0.01 * N), replace=False)
df = pd.concat([df, df.loc[dup_idx]], ignore_index=True).reset_index(drop=True)

# Flags
df['Missing_Flag'] = df.isnull().any(axis=1).astype(int)
df['Duplicate_Flag'] = df.duplicated(subset=['Account_ID','Report_Date','Exposure_Amount'], keep=False).astype(int)

# Save CSV
os.makedirs('/mnt/data', exist_ok=True)
file_path = '/mnt/data/realistic_regulatory_data.csv'
df.to_csv(file_path, index=False)

# Summary
N_total = len(df)
profile = {
    'Total_Records': N_total,
    'Total_Accounts': df['Account_ID'].nunique(),
    'Missing_Percentage': round(df['Missing_Flag'].sum() / N_total * 100, 3),
    'Duplicate_Percentage': round(df['Duplicate_Flag'].sum() / N_total * 100, 3),
    'Labeled_Anomalies': int(df['Is_Anomaly'].sum()),
    'Negative_Exposures': int((df['Exposure_Amount'] <= 0).sum())
}

print("Realistic regulatory-style dataset generated with {} records.".format(N_total))
print("Saved to:", file_path)
print("\nBasic Data Profile:")
for k,v in profile.items():
    print(f"{k}: {v}")

# Save sample top 10
df.head(10).to_csv('/mnt/data/realistic_sample_top10.csv', index=False)

file_path


Realistic regulatory-style dataset generated with 7070 records.
Saved to: /mnt/data/realistic_regulatory_data.csv

Basic Data Profile:
Total_Records: 7070
Total_Accounts: 7000
Missing_Percentage: 53.267
Duplicate_Percentage: 1.98
Labeled_Anomalies: 351
Negative_Exposures: 70


'/mnt/data/realistic_regulatory_data.csv'

In [3]:
# Post-process dataset to recompute Missing_Flag based on critical fields only
import pandas as pd
file_path = '/mnt/data/realistic_regulatory_data.csv'
df = pd.read_csv(file_path, parse_dates=['Report_Date'])

# Define critical fields for DQ: Exposure_Amount, Risk_Weight, Currency, Country_Code
critical_fields = ['Exposure_Amount','Risk_Weight','Currency','Country_Code']

df['Missing_Flag'] = df[critical_fields].isnull().any(axis=1).astype(int)
df['Duplicate_Flag'] = df.duplicated(subset=['Account_ID','Report_Date','Exposure_Amount'], keep=False).astype(int)

# Recompute basic profile
N_total = len(df)
profile = {
    'Total_Records': N_total,
    'Total_Accounts': df['Account_ID'].nunique(),
    'Missing_Percentage': round(df['Missing_Flag'].sum() / N_total * 100, 3),
    'Duplicate_Percentage': round(df['Duplicate_Flag'].sum() / N_total * 100, 3),
    'Labeled_Anomalies': int(df['Is_Anomaly'].sum()),
    'Negative_Exposures': int((df['Exposure_Amount'] <= 0).sum())
}

# Save updated CSV and sample
df.to_csv(file_path, index=False)
df.head(10).to_csv('/mnt/data/realistic_sample_top10.csv', index=False)

print("Recomputed Missing_Flag on critical fields and updated file:", file_path)
print("\nUpdated Basic Data Profile:")
for k,v in profile.items():
    print(f"{k}: {v}")

file_path


Recomputed Missing_Flag on critical fields and updated file: /mnt/data/realistic_regulatory_data.csv

Updated Basic Data Profile:
Total_Records: 7070
Total_Accounts: 7000
Missing_Percentage: 1.429
Duplicate_Percentage: 1.98
Labeled_Anomalies: 351
Negative_Exposures: 70


'/mnt/data/realistic_regulatory_data.csv'

In [4]:
# Running model-comparison on the final realistic_regulatory_data.csv
# This cell trains IsolationForest, Autoencoder, LOF, One-Class SVM on the dataset,
# computes scores, evaluates (ROC AUC), computes SHAP for IsolationForest (sample),
# computes permutation importance for Autoencoder (approx), and saves artifacts.
# Outputs summary metrics below and saves CSVs/plots to /mnt/data.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import shap
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load dataset
csv_path = "/mnt/data/realistic_regulatory_data.csv"
df = pd.read_csv(csv_path, parse_dates=['Report_Date'])
print("Loaded dataset:", csv_path, "shape:", df.shape)

# Select features: numeric + categorical for modelling
numeric_features = ['Exposure_Amount', 'Risk_Weight', 'Capital_Requirement', 'Tenor_Months', 'Transaction_Count']
categorical_features = ['Country_Code', 'Currency', 'Counterparty_Type', 'Source_System', 'Product_Type']

# Prepare labels
if 'Is_Anomaly' in df.columns:
    y = df['Is_Anomaly'].astype(int).values
else:
    y = np.zeros(len(df))

# Fill NA
df[numeric_features] = df[numeric_features].fillna(0)
df[categorical_features] = df[categorical_features].fillna('MISSING')

X = df[numeric_features + categorical_features].copy()

# Train-test split (stratify if labels present)
if y.sum() > 0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
else:
    X_train, X_test, y_train, y_test = train_test_split(X, test_size=0.30, random_state=42)

print("Train/test shapes:", X_train.shape, X_test.shape, "Label counts in test:", np.bincount(y_test))

# Preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
])

preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Save preprocessor
os.makedirs('/mnt/data/models', exist_ok=True)
joblib.dump(preprocessor, '/mnt/data/models/preprocessor_realistic.joblib')

# Isolation Forest
if_model = IsolationForest(n_estimators=200, contamination=0.05, random_state=42)
if_model.fit(X_train_proc)
if_scores_test = -if_model.decision_function(X_test_proc)
if_scores_test = (if_scores_test - if_scores_test.min()) / (if_scores_test.max() - if_scores_test.min())
joblib.dump(if_model, '/mnt/data/models/isolation_forest_realistic.joblib')

# LOF novelty True
lof = LocalOutlierFactor(n_neighbors=35, contamination=0.05, novelty=True)
lof.fit(X_train_proc)
lof_scores_test = -lof.decision_function(X_test_proc)
lof_scores_test = (lof_scores_test - lof_scores_test.min()) / (lof_scores_test.max() - lof_scores_test.min())
joblib.dump(lof, '/mnt/data/models/lof_realistic.joblib')

# One-Class SVM
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
ocsvm.fit(X_train_proc)
ocsvm_scores_test = -ocsvm.decision_function(X_test_proc)
ocsvm_scores_test = (ocsvm_scores_test - ocsvm_scores_test.min()) / (ocsvm_scores_test.max() - ocsvm_scores_test.min())
joblib.dump(ocsvm, '/mnt/data/models/ocsvm_realistic.joblib')

# Autoencoder
input_dim = X_train_proc.shape[1]
encoding_dim = max(8, int(input_dim / 6))

autoencoder = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(encoding_dim*2, activation='relu'),
    layers.Dense(encoding_dim, activation='relu'),
    layers.Dense(encoding_dim*2, activation='relu'),
    layers.Dense(input_dim, activation='linear')
])
autoencoder.compile(optimizer='adam', loss='mse')

history = autoencoder.fit(X_train_proc, X_train_proc, epochs=20, batch_size=256, validation_split=0.1, verbose=0)
X_test_pred = autoencoder.predict(X_test_proc)
recon_error = np.mean(np.square(X_test_proc - X_test_pred), axis=1)
recon_scores_test = (recon_error - recon_error.min()) / (recon_error.max() - recon_error.min())
autoencoder.save('/mnt/data/models/autoencoder_realistic.h5')

# Save training loss plot
plt.figure(figsize=(6,3))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Autoencoder Training Loss')
plt.savefig('/mnt/data/autoencoder_training_loss_realistic.png', bbox_inches='tight', dpi=200)
plt.close()

# Compile scores DataFrame
scores_df = pd.DataFrame({
    'Is_Anomaly': y_test,
    'IF_Score': if_scores_test,
    'LOF_Score': lof_scores_test,
    'OCSVM_Score': ocsvm_scores_test,
    'AE_ReconScore': recon_scores_test
})

scores_df.to_csv('/mnt/data/model_scores_comparison_realistic.csv', index=False)

# Evaluation - ROC AUC
results = {}
if y_test.sum() > 0:
    results['IsolationForest_ROC_AUC'] = roc_auc_score(y_test, if_scores_test)
    results['LOF_ROC_AUC'] = roc_auc_score(y_test, lof_scores_test)
    results['OCSVM_ROC_AUC'] = roc_auc_score(y_test, ocsvm_scores_test)
    results['Autoencoder_ROC_AUC'] = roc_auc_score(y_test, recon_scores_test)
else:
    results['Warning'] = 'No positive labels in test set.'

# Precision@k (top 5%)
k = int(0.05 * len(scores_df))
precisions = {}
for col in ['IF_Score','LOF_Score','OCSVM_Score','AE_ReconScore']:
    topk = scores_df.nlargest(k, col)
    if topk['Is_Anomaly'].sum() == 0:
        precisions[col+'_prec_at_5pct'] = 0.0
    else:
        precisions[col+'_prec_at_5pct'] = topk['Is_Anomaly'].sum() / k

# SHAP for Isolation Forest (use small background sample)
explainer = shap.TreeExplainer(if_model)
n_shap = min(200, X_test_proc.shape[0])
shap_vals = explainer.shap_values(X_test_proc[:n_shap])
np.save('/mnt/data/shap_values_if_realistic.npy', shap_vals)

# Permutation importance for Autoencoder (approx)
base_mse = np.mean(np.square(X_test_proc - X_test_pred), axis=1).mean()
perm_imp = []
for col in range(X_test_proc.shape[1]):
    X_perm = X_test_proc.copy()
    np.random.shuffle(X_perm[:, col])
    pred = autoencoder.predict(X_perm)
    mse_perm = np.mean(np.square(X_perm - pred), axis=1).mean()
    perm_imp.append(mse_perm - base_mse)

# Map feature names
ohe = preprocessor.named_transformers_['cat']
cat_names = list(ohe.get_feature_names_out(categorical_features))
feature_names = numeric_features + cat_names
feature_names = feature_names[:len(perm_imp)]
perm_df = pd.DataFrame({'feature': feature_names, 'perm_importance': perm_imp})
perm_df.sort_values('perm_importance', ascending=False, inplace=True)
perm_df.to_csv('/mnt/data/autoencoder_perm_importance_realistic.csv', index=False)

# Save summary and results
summary = {
    'train_shape': X_train_proc.shape,
    'test_shape': X_test_proc.shape,
    'label_counts_test': int(y_test.sum()),
    'results': results,
    'precisions_top5pct': precisions
}
pd.to_pickle(summary, '/mnt/data/modeling_summary_realistic.pkl')

# Print concise summary
print("=== MODELING SUMMARY ===")
print("Train shape:", X_train_proc.shape, "Test shape:", X_test_proc.shape)
print("Label count (test):", int(y_test.sum()))
print("ROC AUC results (if labels present):")
for k,v in results.items():
    print(f"  {k}: {v}")
print("Precision@5% for models:")
for k,v in precisions.items():
    print(f"  {k}: {v:.3f}")

print("\nArtifacts saved to /mnt/data/:")
print(" - model_scores_comparison_realistic.csv")
print(" - autoencoder_training_loss_realistic.png")
print(" - shap_values_if_realistic.npy")
print(" - autoencoder_perm_importance_realistic.csv")
print(" - models/ (preprocessor + trained models)")
print(" - modeling_summary_realistic.pkl")


Loaded dataset: /mnt/data/realistic_regulatory_data.csv shape: (7070, 17)
Train/test shapes: (4949, 10) (2121, 10) Label counts in test: [2016  105]
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
67/67 ━━━━━━━━━━━━━━━━━

In [5]:
# Data Quality Rule Engine module
# This script reads a regulatory-style CSV, applies deterministic data-quality rules aligned to PRA/EBA/ECB,
# and outputs a flagged dataset and summary KPIs.
# Save this cell as dq_rule_engine.py or run directly in Colab/Jupyter.
#
# Usage example (in Colab):
# 1. Upload / copy your CSV to /content/ or /mnt/data/
# 2. Adjust CSV_PATH below or pass path as argument
# 3. Run the script. It will save dq_report.csv and dq_summary.json in the same directory.
#
# The code is intentionally self-contained and does not require special packages beyond pandas & numpy.

import pandas as pd
import numpy as np
import json
import os
from typing import List, Dict

# ------------------------------ Configuration ------------------------------
# Allowed code-lists and simple business rules. Edit as needed for your environment.
ALLOWED_CURRENCIES = {'INR','USD','GBP','EUR','HKD','SGD'}
ISO_COUNTRIES = {'GB','DE','HK','US','FR','IN','SG','ES','IT','NL'}  # subset used in dataset
CURRENCY_BY_COUNTRY = {
    'IN':'INR','GB':'GBP','DE':'EUR','HK':'HKD','US':'USD','FR':'EUR','SG':'SGD','ES':'EUR','IT':'EUR','NL':'EUR'
}
CRITICAL_FIELDS = ['Exposure_Amount','Risk_Weight','Currency','Country_Code','Counterparty_Type','Report_Date']

# Thresholds / bounds
MIN_EXPOSURE = 0  # exposures must be >= 0
RISK_WEIGHT_MIN, RISK_WEIGHT_MAX = 0.0, 1.0
TENOR_MIN, TENOR_MAX = 1, 360
CAPITAL_TOLERANCE_PCT = 0.05  # 5% tolerance for capital recomputation check

# Duplicate key definition
DUP_KEY = ['Account_ID','Report_Date','Exposure_Amount']

# ------------------------------ Utility functions ------------------------------
def load_data(csv_path: str) -> pd.DataFrame:
    """Load CSV into dataframe and ensure report date is parsed."""
    df = pd.read_csv(csv_path, parse_dates=['Report_Date'], dayfirst=True, low_memory=False)
    return df

def safe_div(a, b):
    try:
        return a / b
    except Exception:
        return np.nan

# ------------------------------ Rule implementations ------------------------------
def check_completeness(df: pd.DataFrame, fields: List[str]=CRITICAL_FIELDS) -> pd.Series:
    """Return binary flag: 1 if any critical field is missing, else 0."""
    return df[fields].isnull().any(axis=1).astype(int)

def check_currency_validity(df: pd.DataFrame) -> pd.Series:
    """Flag records where currency is missing or not in allowed list."""
    return (~df['Currency'].isin(ALLOWED_CURRENCIES)).astype(int)

def check_country_validity(df: pd.DataFrame) -> pd.Series:
    """Flag records with country codes outside ISO_COUNTRIES subset."""
    return (~df['Country_Code'].isin(ISO_COUNTRIES)).astype(int)

def check_currency_country_mismatch(df: pd.DataFrame) -> pd.Series:
    """Flag where country->currency mapping does not match expected mapping (if both present)."""
    def mismatch(row):
        c = row.get('Country_Code')
        cur = row.get('Currency')
        if pd.isna(c) or pd.isna(cur):
            return 0
        expected = CURRENCY_BY_COUNTRY.get(c)
        if expected is None:
            return 0
        return int(cur != expected)
    return df.apply(mismatch, axis=1)

def check_negative_exposure(df: pd.DataFrame) -> pd.Series:
    return (df['Exposure_Amount'] < MIN_EXPOSURE).astype(int)

def check_risk_weight_bounds(df: pd.DataFrame) -> pd.Series:
    return (~df['Risk_Weight'].between(RISK_WEIGHT_MIN, RISK_WEIGHT_MAX)).astype(int)

def check_tenor_bounds(df: pd.DataFrame) -> pd.Series:
    if 'Tenor_Months' in df.columns:
        return (~df['Tenor_Months'].between(TENOR_MIN, TENOR_MAX)).astype(int)
    else:
        return pd.Series(0, index=df.index)

def check_capital_consistency(df: pd.DataFrame) -> pd.Series:
    """Check if Capital_Requirement roughly equals Exposure * RW * 8% (within tolerance)."""
    # Compute expected capital (float) and compare with reported if present
    exp_cap = df['Exposure_Amount'] * df['Risk_Weight'] * 0.08
    reported = df['Capital_Requirement'].fillna(-1)
    # Where reported <=0 treat as inconsistent unless both zero
    # Compute relative diff safely
    rel_diff = (reported - exp_cap).abs() / (exp_cap.replace(0, np.nan).abs())
    # Flag if relative diff > tolerance or reported negative while expected positive
    flag = ((rel_diff > CAPITAL_TOLERANCE_PCT) & (~exp_cap.isnull())).fillna(False)
    # also flag when expected>0 and reported<=0
    flag2 = ((exp_cap > 0) & (reported <= 0))
    return (flag | flag2).astype(int)

def check_duplicates(df: pd.DataFrame, key: List[str]=DUP_KEY) -> pd.Series:
    return df.duplicated(subset=key, keep=False).astype(int)

def check_outlier_simple(df: pd.DataFrame, multiple: float=10.0) -> pd.Series:
    """Simple outlier heuristic: exposure > multiple * median(exposure by country)"""
    med = df.groupby('Country_Code')['Exposure_Amount'].transform('median').replace(0, np.nan)
    return (df['Exposure_Amount'].abs() > multiple * med).fillna(0).astype(int)

# ------------------------------ Main engine ------------------------------
def apply_dq_rules(df: pd.DataFrame) -> pd.DataFrame:
    """Apply all DQ rules and return dataframe with new flag columns and a DQ_Score."""
    df = df.copy()
    # Ensure critical numeric columns exist
    for col in ['Exposure_Amount','Risk_Weight','Capital_Requirement']:
        if col not in df.columns:
            df[col] = np.nan
    # Completeness
    df['DQ_Missing_Flag'] = check_completeness(df)
    # Validity checks
    df['DQ_Invalid_Currency_Flag'] = check_currency_validity(df)
    df['DQ_Invalid_Country_Flag'] = check_country_validity(df)
    df['DQ_Ccy_Ctry_Mismatch_Flag'] = check_currency_country_mismatch(df)
    # Numeric & threshold checks
    df['DQ_Negative_Exposure_Flag'] = check_negative_exposure(df)
    df['DQ_RiskWeight_Flag'] = check_risk_weight_bounds(df)
    df['DQ_Tenor_Flag'] = check_tenor_bounds(df)
    df['DQ_Capital_Inconsistency_Flag'] = check_capital_consistency(df)
    # Duplicates & outliers
    df['DQ_Duplicate_Flag'] = check_duplicates(df)
    df['DQ_Exposure_Outlier_Flag'] = check_outlier_simple(df)
    # Combine into an overall DQ Rule-based score (weighted sum)
    # Weights are configurable — here we use equal weight for simplicity
    flag_cols = [c for c in df.columns if c.startswith('DQ_') and c.endswith('_Flag')]
    df['DQ_Rule_Score'] = df[flag_cols].sum(axis=1).astype(int)  # simple count of flags
    # Binary rule-based anomaly: if any rule flag or rule score > 0
    df['DQ_Rule_Anomaly'] = (df['DQ_Rule_Score'] > 0).astype(int)
    return df

def compute_dq_kpis(df: pd.DataFrame) -> Dict[str, float]:
    """Compute summary KPIs for the dataset based on applied DQ flags."""
    total = len(df)
    kpi = {
        'Total_Records': int(total),
        'Missing_Percentage': float(df['DQ_Missing_Flag'].sum() / total * 100),
        'Duplicate_Percentage': float(df['DQ_Duplicate_Flag'].sum() / total * 100),
        'Invalid_Currency_Percentage': float(df['DQ_Invalid_Currency_Flag'].sum() / total * 100),
        'Invalid_Country_Percentage': float(df['DQ_Invalid_Country_Flag'].sum() / total * 100),
        'Negative_Exposure_Count': int(df['DQ_Negative_Exposure_Flag'].sum()),
        'Capital_Inconsistency_Count': int(df['DQ_Capital_Inconsistency_Flag'].sum()),
        'Exposure_Outlier_Percentage': float(df['DQ_Exposure_Outlier_Flag'].sum() / total * 100),
        'Rule_Anomaly_Percentage': float(df['DQ_Rule_Anomaly'].sum() / total * 100),
    }
    return kpi

def save_outputs(df: pd.DataFrame, out_csv: str, summary_json: str=None):
    """Save flagged dataframe and optional summary JSON to disk."""
    df.to_csv(out_csv, index=False)
    print(f"Saved flagged dataset to: {out_csv}")
    if summary_json:
        kpi = compute_dq_kpis(df)
        with open(summary_json, 'w') as f:
            json.dump(kpi, f, indent=2)
        print(f"Saved DQ summary to: {summary_json}")
        return kpi
    return None

# ------------------------------ Example script entrypoint ------------------------------
if __name__ == '__main__':
    # Default paths (edit if running locally)
    INPUT_CSV = '/mnt/data/realistic_regulatory_data.csv'
    OUTPUT_CSV = '/mnt/data/dq_report_realistic.csv'
    SUMMARY_JSON = '/mnt/data/dq_summary_realistic.json'

    if not os.path.exists(INPUT_CSV):
        print("Input CSV not found at", INPUT_CSV)
    else:
        df_in = load_data(INPUT_CSV)
        df_flagged = apply_dq_rules(df_in)
        kpi = save_outputs(df_flagged, OUTPUT_CSV, SUMMARY_JSON)
        print("\nDQ KPIs:")
        print(json.dumps(kpi, indent=2))


Saved flagged dataset to: /mnt/data/dq_report_realistic.csv
Saved DQ summary to: /mnt/data/dq_summary_realistic.json

DQ KPIs:
{
  "Total_Records": 7070,
  "Missing_Percentage": 1.4285714285714286,
  "Duplicate_Percentage": 1.9801980198019802,
  "Invalid_Currency_Percentage": 1.5841584158415842,
  "Invalid_Country_Percentage": 0.32531824611032534,
  "Negative_Exposure_Count": 70,
  "Capital_Inconsistency_Count": 0,
  "Exposure_Outlier_Percentage": 4.837340876944837,
  "Rule_Anomaly_Percentage": 9.886845827439886
}
